In [2]:
# import sys
# IN_COLAB = 'google.colab' in sys.modules

# if IN_COLAB:
#     !pip install deriva
#     !pip install bdbag
#     !pip install --upgrade --force pydantic
#     !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml

In [3]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [4]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [5]:

from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-06-30 16:07:20,583 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-30 16:07:20,584 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-06-30 16:07:20,586 - INFO - Setting up RefreshTokenAuthorizer with auth_client=[instance:140152729768896]
2024-06-30 16:07:20,586 - INFO - Setting up a RenewingAuthorizer. It will use an auth type of Bearer and can handle 401s.
2024-06-30 16:07:20,587 - INFO - RenewingAuthorizer will start by using access_token with hash "4b84a6ba5a19d3fec1774ba585ebd6379414a44f7852c6d9faf9763462506e6e"
2024-06-30 16:07:20,588 - INFO - Executing token refresh without client credentials
2024-06-30 16:07:20,589 - INFO - Fetching new token from Globus Auth
2024-06-30 16:07:20,978 - INFO - request done (success)
2024-06-30 16:07:20,979 - INFO - RenewingAuthorizer.

You are already logged in.


In [6]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid = "2-C8W6" # rid
# Change the confi_file with bag_url=["minid: train", "minid: Valid", "minid: test"]



In [7]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-06-30 16:07:21,164 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-30 16:07:21,165 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [8]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-06-30 16:07:23,158 - INFO - File [/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_training_LACDHS_quality_sreenidhi_june_27_2024.json] transfer successful. 0.95 KB transferred. Elapsed time: 0:00:00.000078.
2024-06-30 16:07:23,159 - INFO - Verifying MD5 checksum for downloaded file [/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_training_LACDHS_quality_sreenidhi_june_27_2024.json]
2024-06-30 16:07:23,177 - INFO - Configuration validation successful!
2024-06-30 16:07:32,560 - INFO - File [/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json] transfer successful. 0.69 KB transferred. Elapsed time: 0:00:00.000117.
2024-06-30 16:07:32,561 - INFO - Verifying SHA256 checksum for downloaded file [/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json]
2024-06-30 16:07:32,902 - INFO - File [

{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/sreenidhi/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'VGG19_Catalog_Model_LACDHS_quality_training',
    'rid': '2-C8WA'}],
  'Execution_Asset_Type': [{'name': 'VGG19_Catalog_Model_LACDHS_quality_training',
    'rid': '2-C8WC'}]},
 'execution_rid': '2-C92M',
 'workflow_rid': '2-C8WE',
 'bag_paths': [PosixPath('/data/2-277G_6aa1a6861eee5a79bce4bf071065355f95a066c2a1ff326089d43048a7e0f185/Dataset_2-277G'),
  PosixPath('/data/2-277J_81c873a311aa6a67cf2eef44bd9056cb19181b299a6e44327ea3553616f18725/Dataset_2-277J'),
  PosixPath('/data/2-277M_8c4b855c2752e098580a5bb0d1b63a8cedde4462805fe74cddc912a72fb39963/Dataset_2-277M')],
 'assets_paths': [PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json'),
  PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/train_no_optic_disc_image_ids.csv'),
  PosixPath('/data/sreenidhi/EyeAI_working/Exec

In [9]:
configuration_records

ConfigurationRecord(caching_dir=PosixPath('/data'), working_dir=PosixPath('/data/sreenidhi/EyeAI_working'), vocabs={'Workflow_Type': [Term(name='VGG19_Catalog_Model_LACDHS_quality_training', rid='2-C8WA')], 'Execution_Asset_Type': [Term(name='VGG19_Catalog_Model_LACDHS_quality_training', rid='2-C8WC')]}, execution_rid='2-C92M', workflow_rid='2-C8WE', bag_paths=[PosixPath('/data/2-277G_6aa1a6861eee5a79bce4bf071065355f95a066c2a1ff326089d43048a7e0f185/Dataset_2-277G'), PosixPath('/data/2-277J_81c873a311aa6a67cf2eef44bd9056cb19181b299a6e44327ea3553616f18725/Dataset_2-277J'), PosixPath('/data/2-277M_8c4b855c2752e098580a5bb0d1b63a8cedde4462805fe74cddc912a72fb39963/Dataset_2-277M')], assets_paths=[PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json'), PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/train_no_optic_disc_image_ids.csv'), PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/valid_no_optic_di

# 5 classes and Full fundus converted to 2 classes simialr to van quality labels good and bad (first 3 classes are good and last 2 are bad)

In [12]:



# import os
# import shutil
# import pandas as pd
# from pathlib import Path

# def create_LACDHS_quality_dataset(train_dir: str, validation_dir: str, test_dir: str, output_dir: str, exclude_train: list = [], exclude_valid: list = []) -> tuple:
#     """
#     Creates a dataset for LACDHS image quality classification by organizing images into train, valid, and test folders
#     based on their Subject_image_quality from the subject table.

#     Parameters:
#     - train_dir (str): Path to the raw train dataset bag.
#     - validation_dir (str): Path to the raw validation dataset bag.
#     - test_dir (str): Path to the raw test dataset bag.
#     - output_dir (str): Path to the output directory where the organized dataset will be created.
#     - exclude_train (list): List of image RIDs to exclude from the train set.
#     - exclude_valid (list): List of image RIDs to exclude from the validation set.

#     Returns:
#     - tuple: A tuple containing the paths to the train, validation, and test directories.
#     """
#     def process_dataset(bag_path: str, output_subdir: str, exclude_list: list = []):
#         image_csv_path = os.path.join(bag_path, 'data', 'Image.csv')
#         observation_csv_path = os.path.join(bag_path, 'data', 'Observation.csv')
#         subject_csv_path = os.path.join(bag_path, 'data', 'Subject.csv')

#         image_df = pd.read_csv(image_csv_path)
#         observation_df = pd.read_csv(observation_csv_path)
#         subject_df = pd.read_csv(subject_csv_path)

#         # Merge dataframes
#         merged_df = image_df.merge(observation_df, left_on='Observation', right_on='RID')
#         merged_df = merged_df.merge(subject_df, left_on='Subject', right_on='RID')

#         # Filter for specific quality categories
#         quality_categories = ['69XP', '69XJ', '69XT', '69XM', '69XR']
#         merged_df = merged_df[merged_df['Subject_image_quality'].isin(quality_categories)]

#         image_root_path = os.path.join(bag_path, 'data', 'assets', 'Image')

#         for _, row in merged_df.iterrows():
#             if row['RID_x'] not in exclude_list:  # RID_x is the Image RID
#                 quality = row['Subject_image_quality']
#                 filename = row['Filename']
#                 src_path = os.path.join(image_root_path, filename)
#                 dst_dir = os.path.join(output_dir, output_subdir, quality)
#                 os.makedirs(dst_dir, exist_ok=True)
#                 dst_path = os.path.join(dst_dir, filename)
#                 shutil.copy2(src_path, dst_path)

#     # Process train dataset
#     process_dataset(train_dir, 'train', exclude_train)

#     # Process validation dataset
#     process_dataset(validation_dir, 'valid', exclude_valid)

#     # Process test dataset
#     process_dataset(test_dir, 'test')

#     train_path = os.path.join(output_dir, 'train')
#     valid_path = os.path.join(output_dir, 'valid')
#     test_path = os.path.join(output_dir, 'test')

#     return train_path, valid_path, test_path


import os
import shutil
import pandas as pd
from pathlib import Path
from tqdm import tqdm

def create_LACDHS_binary_quality_dataset(train_dir: str, validation_dir: str, test_dir: str, output_dir: str) -> tuple:
    """
    Creates a binary dataset for LACDHS image quality classification by organizing images into train, valid, and test folders
    based on their Subject_image_quality from the subject table, mapped to binary "Good" and "Bad" categories.
    
    Parameters:
    - train_dir (str): Path to the raw train dataset bag.
    - validation_dir (str): Path to the raw validation dataset bag.
    - test_dir (str): Path to the raw test dataset bag.
    - output_dir (str): Path to the output directory where the organized dataset will be created.
    
    Returns:
    - tuple: A tuple containing the paths to the train, validation, and test directories.
    """
    def process_dataset(bag_path: str, output_subdir: str):
        image_csv_path = os.path.join(bag_path, 'data', 'Image.csv')
        observation_csv_path = os.path.join(bag_path, 'data', 'Observation.csv')
        subject_csv_path = os.path.join(bag_path, 'data', 'Subject.csv')
        
        image_df = pd.read_csv(image_csv_path)
        observation_df = pd.read_csv(observation_csv_path)
        subject_df = pd.read_csv(subject_csv_path)
        
        # Merge dataframes
        merged_df = image_df.merge(observation_df, left_on='Observation', right_on='RID')
        merged_df = merged_df.merge(subject_df, left_on='Subject', right_on='RID')
        
        # Define quality mapping
        quality_mapping = { 
            '69XT': '690J',  # Excellent #690J eye-ai:690J Good
            '69XP': '690J',  # Adequate #690J eye-ai:690J Good
            '69XJ': '690J',  # Good #690J eye-ai:690J Good
            '69XM': '692J',   # Insufficient for Full Interpretation # 692J eye-ai:692J Bad
            '69XR': '692J'    # Insufficient for Any Interpretation # 692J eye-ai:692J Bad
        }
        
        # Filter for specific quality categories and map to binary categories
        merged_df = merged_df[merged_df['Subject_image_quality'].isin(quality_mapping.keys())]
        merged_df['binary_quality'] = merged_df['Subject_image_quality'].map(quality_mapping)
        
        image_root_path = os.path.join(bag_path, 'data', 'assets', 'Image')
        
        # Using tqdm to show progress
        for _, row in tqdm(merged_df.iterrows(), total=len(merged_df), desc=f"Processing {output_subdir}"):
            binary_quality = row['binary_quality']
            filename = row['Filename']
            src_path = os.path.join(image_root_path, filename)
            dst_dir = os.path.join(output_dir, output_subdir, binary_quality)
            os.makedirs(dst_dir, exist_ok=True)
            dst_path = os.path.join(dst_dir, filename)
            shutil.copy2(src_path, dst_path)

    print("Starting dataset creation...")
    
    # Process train dataset
    process_dataset(train_dir, 'train')
    
    # Process validation dataset
    process_dataset(validation_dir, 'valid')
    
    # Process test dataset
    process_dataset(test_dir, 'test')

    train_path = os.path.join(output_dir, 'train')
    valid_path = os.path.join(output_dir, 'valid')
    test_path = os.path.join(output_dir, 'test')

    print("Dataset creation completed.")
    return train_path, valid_path, test_path



In [13]:
configuration_records.working_dir

PosixPath('/data/sreenidhi/EyeAI_working')

In [15]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)
train_dir = configuration_records.bag_paths[0] # path to the raw train dataset
validation_dir = configuration_records.bag_paths[1]
test_dir = configuration_records.bag_paths[2]

# Call the create_LACDHS_binary_quality_dataset function
train_path, valid_path, test_path = create_LACDHS_binary_quality_dataset(
    train_dir=str(configuration_records.bag_paths[0]),
    validation_dir=str(configuration_records.bag_paths[1]),
    test_dir=str(configuration_records.bag_paths[2]),
    output_dir=str(configuration_records.working_dir),
)

# Print the paths to verify
print("Train dataset path:", train_path)
print("Validation dataset path:", valid_path)
print("Test dataset path:", test_path)

Starting dataset creation...


Processing test: 100%|██████████| 3144/3144 [00:29<00:00, 105.12it/s]

Dataset creation completed.
Train dataset path: /data/sreenidhi/EyeAI_working/train
Validation dataset path: /data/sreenidhi/EyeAI_working/valid
Test dataset path: /data/sreenidhi/EyeAI_working/test


In [16]:
import os

def count_files(directory):
    return len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])

def analyze_lacdhs_angle_dataset(base_path):
    main_folders = ['train', 'valid', 'test']
    
    for main_folder in main_folders:
        main_folder_path = os.path.join(base_path, main_folder)
        if not os.path.exists(main_folder_path):
            print(f"{main_folder} folder not found")
            continue
        
        print(f"\nAnalyzing {main_folder} folder:")
        
        total_files = 0
        for angle_folder in os.listdir(main_folder_path):
            angle_folder_path = os.path.join(main_folder_path, angle_folder)
            if os.path.isdir(angle_folder_path):
                file_count = count_files(angle_folder_path)
                print(f"  {angle_folder}: {file_count} images")
                total_files += file_count
        
        print(f"Total images in {main_folder}: {total_files}")

# Usage
base_path = "/data/sreenidhi/EyeAI_working/"
analyze_lacdhs_angle_dataset(base_path)






Analyzing train folder:
  690J: 25091 images
  692J: 1231 images
Total images in train: 26322

Analyzing valid folder:
  690J: 8234 images
  692J: 441 images
Total images in valid: 8675

Analyzing test folder:
  690J: 3015 images
  692J: 129 images
Total images in test: 3144


In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image

def visualize_lacdhs_quality_dataset(base_path, samples_per_angle=6):
    main_folders = ['train', 'valid', 'test']
    
    for main_folder in main_folders:
        main_folder_path = os.path.join(base_path, main_folder)
        if not os.path.exists(main_folder_path):
            print(f"{main_folder} folder not found")
            continue
        
        print(f"\nVisualizing samples from {main_folder} folder:")
        
        angle_folders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))]
        
        # Calculate grid size
        n_angles = len(angle_folders)
        n_cols = samples_per_angle
        n_rows = n_angles
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*3, n_rows*3.5))
        fig.suptitle(f'Sample Images from {main_folder.capitalize()} Set', fontsize=16)
        
        for i, angle_folder in enumerate(angle_folders):
            angle_folder_path = os.path.join(main_folder_path, angle_folder)
            image_files = [f for f in os.listdir(angle_folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            if len(image_files) < samples_per_angle:
                print(f"Warning: Not enough images in {angle_folder}. Using all available images.")
                selected_files = image_files
            else:
                selected_files = random.sample(image_files, samples_per_angle)
            
            for j, image_file in enumerate(selected_files):
                img_path = os.path.join(angle_folder_path, image_file)
                img = Image.open(img_path)
                axes[i, j].imshow(img)
                axes[i, j].axis('off')
                
                # Add image filename as title for each subplot
                axes[i, j].set_title(image_file, fontsize=8)
                
                if j == 0:
                    axes[i, j].set_ylabel(angle_folder, rotation=0, labelpad=40, va='center', fontsize=10)
        
        plt.tight_layout()
        plt.subplots_adjust(top=0.95, bottom=0.05, left=0.2, right=0.98)
        plt.show()
        
        # Print confirmation of angles
        print(f"Angles in {main_folder} set:")
        for angle in angle_folders:
            print(f"  - {angle}")

# Usage
base_path = "/data/sreenidhi/EyeAI_working/"
# visualize_lacdhs_quality_dataset(base_path)

In [18]:

output_path = str(EA.working_dir) + "/Execution_Assets/" + configuration_records.vocabs['Execution_Asset_Type'][0].name
os.mkdir(output_path)

In [19]:
output_path

'/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_Model_LACDHS_quality_training'

In [20]:
best_hyper_parameters_json_path = str(configuration_records.assets_paths[0])

In [21]:
best_hyper_parameters_json_path

'/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json'

In [22]:
import json

# Open and load the JSON file
with open(best_hyper_parameters_json_path, 'r') as file:
    data = json.load(file)

# Print the contents of the JSON file
print(json.dumps(data, indent=4))

{
    "rotation_range": -6,
    "width_shift_range": 0.049283662164352315,
    "height_shift_range": 0.062129040368351915,
    "horizontal_flip": true,
    "vertical_flip": true,
    "zoom_range": -0.03493437617512693,
    "brightness_range": 0.016808387649284325,
    "use_class_weights": true,
    "pooling": "global_average",
    "dense_layers": 2,
    "units_layer_0": 512,
    "activation_func_0": "sigmoid",
    "batch_norm_0": true,
    "dropout_0": 0.10646478371824658,
    "units_layer_1": 64,
    "activation_func_1": "relu",
    "batch_norm_1": false,
    "dropout_1": 0.2830490167548361,
    "fine_tune_at": 0,
    "fine_tuning_learning_rate_adam": 1.1688327470992886e-05,
    "batch_size": 32
}


In [24]:
# @title Execute Training algorithm

from eye_ai.models.vgg19_lacdhs_quality_train import main

with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  main(train_path=train_path,
       valid_path=valid_path, 
       output_path=output_path,
       best_hyperparameters_json_path=best_hyper_parameters_json_path,
       model_name="VGG19_Catalog_LAC_DHS_Quality_Trained_model_June_30_2024"
      )


Found 26322 images belonging to 2 classes.
Found 8675 images belonging to 2 classes.
train_generator.class_indices :  {'692J': 0, '690J': 1}
validation_generator.class_indices :  {'692J': 0, '690J': 1}


2024-06-30 16:24:08.916959: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-30 16:24:09.332117: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-30 16:24:09.335854: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/100


2024-06-30 16:24:16.602824: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-06-30 16:24:23.061108: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f76d134eae0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-30 16:24:23.062000: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2024-06-30 16:24:23.093368: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1719789863.326783   23642 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


823/823 [==============================] - 621s 728ms/step - loss: 0.6069 - roc_auc_score: 0.7569 - f1_score_normal: 0.7355 - accuracy_score: 0.6014 - val_loss: 0.8830 - val_roc_auc_score: 0.8164 - val_f1_score_normal: 0.6400 - val_accuracy_score: 0.5006
Epoch 2/100
823/823 [==============================] - 607s 735ms/step - loss: 0.5555 - roc_auc_score: 0.8033 - f1_score_normal: 0.7710 - accuracy_score: 0.6454 - val_loss: 1.6207 - val_roc_auc_score: 0.8386 - val_f1_score_normal: 0.2918 - val_accuracy_score: 0.2169
Epoch 3/100
823/823 [==============================] - 600s 727ms/step - loss: 0.5335 - roc_auc_score: 0.8185 - f1_score_normal: 0.7765 - accuracy_score: 0.6523 - val_loss: 0.2094 - val_roc_auc_score: 0.8375 - val_f1_score_normal: 0.9472 - val_accuracy_score: 0.9032
Epoch 4/100
823/823 [==============================] - 613s 740ms/step - loss: 0.5147 - roc_auc_score: 0.8303 - f1_score_normal: 0.8048 - accuracy_score: 0.6892 - val_loss: 0.5968 - val_roc_auc_score: 0.8433 - v

/home/sreenidhi/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
2024-06-30 19:57:59,030 - INFO - VGG19_Catalog_LAC_DHS_Quality_Trained_model_June_30_2024 Model trained, Model and training history are saved successfully.


In [25]:
data

{'rotation_range': -6,
 'width_shift_range': 0.049283662164352315,
 'height_shift_range': 0.062129040368351915,
 'horizontal_flip': True,
 'vertical_flip': True,
 'zoom_range': -0.03493437617512693,
 'brightness_range': 0.016808387649284325,
 'use_class_weights': True,
 'pooling': 'global_average',
 'dense_layers': 2,
 'units_layer_0': 512,
 'activation_func_0': 'sigmoid',
 'batch_norm_0': True,
 'dropout_0': 0.10646478371824658,
 'units_layer_1': 64,
 'activation_func_1': 'relu',
 'batch_norm_1': False,
 'dropout_1': 0.2830490167548361,
 'fine_tune_at': 0,
 'fine_tuning_learning_rate_adam': 1.1688327470992886e-05,
 'batch_size': 32}

In [31]:
# @title Execute Evaluation algorithm

from eye_ai.models.vgg19_lacdhs_quality_predict import predict_and_evaluate
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
    predict_and_evaluate(
        model_path=output_path + '/VGG19_Catalog_LAC_DHS_Quality_Trained_model_June_30_2024.h5',
        image_path=test_path,
        output_dir=output_path,
        best_hyperparameters_json_path=best_hyper_parameters_json_path
    )

Found 3144 images belonging to 2 classes.
99/99 [==============================] - 120s 1s/step


2024-06-30 22:19:10,239 - INFO - 
Evaluation Metrics:
2024-06-30 22:19:10,240 - INFO - Accuracy: 0.8584605597964376
2024-06-30 22:19:10,240 - INFO - Precision: 0.9838102409638554
2024-06-30 22:19:10,241 - INFO - Recall: 0.8666666666666667
2024-06-30 22:19:10,241 - INFO - F1 Score: 0.9215305942514548
2024-06-30 22:19:10,241 - INFO - ROC AUC: 0.8578477123426794
2024-06-30 22:19:10,258 - INFO - Classification Report:
2024-06-30 22:19:10,259 - INFO - {
  "692J": {
    "precision": 0.1762295081967213,
    "recall": 0.6666666666666666,
    "f1-score": 0.2787682333873582,
    "support": 129.0
  },
  "690J": {
    "precision": 0.9838102409638554,
    "recall": 0.8666666666666667,
    "f1-score": 0.9215305942514548,
    "support": 3015.0
  },
  "accuracy": 0.8584605597964376,
  "macro avg": {
    "precision": 0.5800198745802884,
    "recall": 0.7666666666666666,
    "f1-score": 0.6001494138194065,
    "support": 3144.0
  },
  "weighted avg": {
    "precision": 0.9506747719667308,
    "recall": 


Evaluation Metrics:
Accuracy: 0.8584605597964376
Precision: 0.9838102409638554
Recall: 0.8666666666666667
F1 Score: 0.9215305942514548
ROC AUC: 0.8578477123426794

Classification Report:
              precision    recall  f1-score   support

        692J       0.18      0.67      0.28       129
        690J       0.98      0.87      0.92      3015

    accuracy                           0.86      3144
   macro avg       0.58      0.77      0.60      3144
weighted avg       0.95      0.86      0.90      3144

Predictions saved to quality_predictions.csv
Confusion matrix saved as confusion_matrix.png
ROC curve saved as roc_curve.png


In [32]:
# # @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, True)

2024-06-30 22:20:22,992 - INFO - Initializing uploader: GenericUploader v1.7.1 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2024-06-30 22:20:22,994 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-30 22:20:22,995 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-06-30 22:20:23,033 - INFO - Checking for updated configuration...
2024-06-30 22:20:23,150 - INFO - Updated configuration found.
2024-06-30 22:20:23,152 - INFO - Scanning files in directory [/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_Model_LACDHS_quality_training]...
2024-06-30 22:20:23,156 - INFO - Including file: [/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_Model_LACDHS_quality_training/VGG19_Catalog_LAC_DHS_Quality_Trained_model_June_30_2024.h5].
2024-06-